<a href="https://colab.research.google.com/github/aparnashenoy/ml/blob/master/version13_speed_model_with_regression__testdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
path = '/gdrive/My Drive/'

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
! pip install pygeohash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from pandas import*
import datetime
import glob
from os import listdir
from os.path import isfile, join

# Data Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data pre-processing libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn. metrics import accuracy_score
import pygeohash as gh

In [ ]:
import os, zipfile

dir_name = path + 'zip1'

extension = ".zip"



os.chdir(path+'zip1') # change directory from working dir to dir with files

for item in os.listdir(dir_name): 
  if item.endswith(extension): 
    file_name = os.path.abspath(item) # get full path of files
    zip_ref = zipfile.ZipFile(file_name) # create zipfile object
    zip_ref.extractall(path + '/csv1') # extract file to dir

zip_ref.close() # close file

#os.remove(file_name) # de

In [ ]:
file_list = glob.glob(path + '/csv1/*.*')
print(file_list)

['/gdrive/My Drive//csv1/AIS_2021_01_23.csv', '/gdrive/My Drive//csv1/AIS_2021_01_25.csv', '/gdrive/My Drive//csv1/AIS_2021_01_26.csv', '/gdrive/My Drive//csv1/AIS_2021_01_27.csv', '/gdrive/My Drive//csv1/AIS_2021_01_28.csv', '/gdrive/My Drive//csv1/AIS_2021_01_29.csv']


In [ ]:
my_list = []
new_path = path + '/csv1/*.*'
for file in glob.glob(new_path):
  print(file)
  a = pd.read_csv(file)
  my_list.append(a)


/gdrive/My Drive//csv1/AIS_2021_01_23.csv
/gdrive/My Drive//csv1/AIS_2021_01_25.csv
/gdrive/My Drive//csv1/AIS_2021_01_26.csv
/gdrive/My Drive//csv1/AIS_2021_01_27.csv
/gdrive/My Drive//csv1/AIS_2021_01_28.csv
/gdrive/My Drive//csv1/AIS_2021_01_29.csv


In [ ]:
def drop_missing_values(data):
  data.isna().mean().round(4)*100
  data.dropna(inplace=True)
  data.isna().sum()
  data.drop_duplicates()

In [ ]:
def drop_unwanted_columns(data):
  data.drop(columns=['Heading',
       'VesselName', 'IMO', 'CallSign', 'Status', 'Length',
       'Width', 'Draft', 'Cargo', 'TransceiverClass'], axis = 1, inplace=True)
  


In [ ]:
def drop_invalid_mmsi(data):
  
  invalid_entries=[111111111,222222222,333333333,444444444,555555555,666666666,777777777,888888888,999999999, 123456789, 987654321]
  invalid = data[data.MMSI.isin(invalid_entries)]
  data.drop(invalid.index, axis=0, inplace=True)
  #print(data)




In [ ]:
def invalid_mmsi_length(data):
  data['MMSI_length']= data['MMSI'].map(str).apply(len)
  data = data[data['MMSI_length']==9]
  data.drop('MMSI_length', axis=1, inplace=True)

In [ ]:
def zero_sog(data):
  index_names=data[data['SOG']==0.0].index
  data.drop(index_names, inplace=True)

In [ ]:
def vesseltype_categories(data):
  data.dropna(subset = ["VesselType"], axis = 0, inplace=True)
  data['VesselType'] = data['VesselType'].apply(np.int64)
  data["VesselType"].replace({0: "Not Available", 30: "Fishing", 35:"Military", 
                          (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,23,24,25,26,27,28,29,33,34, 
                           38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59, 90,
                           91,92,93,94,95,96,97,98,99):'Other',
                          (60,61,62,63,64,65,66,67,68,69):'Passenger',
                          (70,71,72,73,74,75,76,77,78,79):'Cargo',
                          (36,37):'Pleasure Craft/Sailing',
                          (80,81,82,83,84,85,86,87,88,89):'Tanker',
                          (21,22,31,32,52):'Tug Tow'}, inplace=True)
  data.VesselType = data.VesselType.map({"Not Available" : 0, "Fishing" : 1,
                                       "Military" : 2, "Other" : 3, "Passenger" : 4, "Cargo" : 5,
                                       "Pleasure Craft/Sailing" : 6, "Tanker" : 7, "Tug Tow" : 8})
  #label_encoder = LabelEncoder()
  #data['VesselType_labeled']=label_encoder.fit_transform(data['VesselType'])
  

In [ ]:
#def label_encode(data):
 # label_encoder = LabelEncoder()
  #data['VesselType_labeled']=label_encoder.fit_transform(data['VesselType'])


In [ ]:
def course_direction(data):
  def categorize(data):
    if data['COG']<22.5:
       return 'N'
    if data['COG']>=22.5 and data['COG']<67.5:
      return 'NE'
    if data['COG']>=67.5 and data['COG']<112.5:
      return 'E'
    if data['COG']>=112.5 and data['COG']<157.5:
      return 'SE'
    if data['COG']>=157.5 and data['COG']<202.5:
      return 'S'
    if data['COG']>=202.5 and data['COG']<247.5:
      return 'SW'
    if data['COG']>=247.5 and data['COG']<292.5:
      return 'W'
    if data['COG']>=292.5 and data['COG']<337.5:
      return 'NW'
    if data['COG']>=337.5:
      return 'N'
  data['COG_direction'] = data.apply(lambda data: categorize(data), axis=1)


In [ ]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371000):
  if to_radians:
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
  a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
  return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [ ]:
def apply_haversine(data):
  data= data.sort_values(['MMSI', 'BaseDateTime'], ascending = True)
  data['distance'] = np.concatenate(data.groupby('MMSI').apply(lambda x: haversine(x['LAT'], x['LON'], x['LAT'].shift(), x['LON'].shift())).values)
  data['distance'] = data['distance'].fillna(0)
  data['dist_diff'] = data['distance'].diff()
  indexNames = data[ data['dist_diff'] <= 500 ].index # Get names of indexes for which column dist_diff has value less than or equal to 1
  data.drop(indexNames , inplace=True) # Delete these row indexes from dataFrame
  data['dist_diff'] = data['dist_diff'].fillna(0)

In [ ]:
def geohash(data):
  data['geohash']=data.apply(lambda x: gh.encode(x.LAT, x.LON, precision=5), axis=1)

In [ ]:
def Eda(data):
  drop_missing_values(data)
  drop_unwanted_columns(data)
  drop_invalid_mmsi(data)
  zero_sog(data)
  vesseltype_categories(data)
  #label_encode(data)
  geohash(data)
  course_direction(data)
  apply_haversine(data)
  print(data)


def readFiles():  
  length = len(my_list)
  for i in range(length):
    Eda(my_list[i])
readFiles()

#def readFiles():
 # length = len(my_list)
 # i = 0
  #while i < length:
     #Eda(my_list[i])    
  #i = i+1
#readFiles()

        MMSI         BaseDateTime       LAT       LON   SOG  COG  VesselType  \
0  209729000  2021-09-30T00:51:31  27.77118 -82.52303  12.9  180           5   
1  209729000  2021-09-30T00:56:21  27.75374 -82.52413  13.0  198           5   
2  209729000  2021-09-30T00:35:40  27.80131 -82.47307  13.3  249           5   
3  209729000  2021-09-30T01:00:01  27.74131 -82.52868  12.9  197           5   
4  209729000  2021-09-30T00:52:50  27.76644 -82.52305  13.0  180           5   
5  209729000  2021-09-30T00:58:41  27.74581 -82.52707  13.0  198           5   
6  209729000  2021-09-30T00:54:01  27.76212 -82.52299  13.0  180           5   

  geohash COG_direction  
0   dhvmf             S  
1   dhvmf             S  
2   dhvq5             W  
3   dhvmf             S  
4   dhvmf             S  
5   dhvmf             S  
6   dhvmf             S  
        MMSI         BaseDateTime       LAT       LON   SOG  COG  VesselType  \
0  209729000  2021-09-30T00:33:21  27.80428 -82.46408  13.3  246       

In [ ]:


folderpath = path+ "backup1"
os.mkdir(folderpath)
for i, df in enumerate(my_list, 1):
    filename = "AIS_2022_01_{}.csv".format(i)
    filepath = os.path.join(folderpath, filename)
    df.to_csv(filepath, index=False)
  

 

In [ ]:
new_file_list = glob.glob(path + '/backup1/*.csv')
print(new_file_list)

['/gdrive/My Drive//backup1/AIS_2022_01_1.csv', '/gdrive/My Drive//backup1/AIS_2022_01_2.csv', '/gdrive/My Drive//backup1/AIS_2022_01_3.csv', '/gdrive/My Drive//backup1/AIS_2022_01_4.csv', '/gdrive/My Drive//backup1/AIS_2022_01_5.csv', '/gdrive/My Drive//backup1/AIS_2022_01_6.csv']


In [ ]:
df_list = (pd.read_csv(file) for file in new_file_list)

In [ ]:
big_df = pd.concat(map(pd.read_csv, glob.glob(path + '/backup1/*.csv')))

In [ ]:
grouped_big_df = big_df.groupby(['geohash', 'VesselType', 'LAT', 'LON', 'SOG']).agg({'VesselType':'count'})
grouped_big_df.columns = ['count']
grouped_big_df = grouped_big_df.reset_index()
grouped_big_df.head(100)

,geohash,VesselType,LAT,LON,SOG,count
0,dhvmf,5,27.74131,-82.52868,12.9,1
1,dhvmf,5,27.74581,-82.52707,13.0,1
2,dhvmf,5,27.75374,-82.52413,13.0,1
3,dhvmf,5,27.76212,-82.52299,13.0,1
4,dhvmf,5,27.76644,-82.52305,13.0,1
5,dhvmf,5,27.77118,-82.52303,12.9,1
6,dhvq4,5,27.78906,-82.50863,13.6,2
7,dhvq4,5,27.79221,-82.49941,13.5,2
8,dhvq4,5,27.79358,-82.49541,13.5,2
9,dhvq4,5,27.79520,-82.49082,13.5,2


In [ ]:
def my_agg(x):
  names = {'weighted_average_SOG': (x['count']*x['SOG']).sum()/x['count'].sum()}
  return pd.Series(names, index=['weighted_average_SOG'])

In [ ]:
wt_grouped_big_df = grouped_big_df.groupby(['geohash', 'VesselType', 'LAT', 'LON', 'SOG']).apply(my_agg)
df = grouped_big_df.merge(wt_grouped_big_df, on = ['geohash', 'VesselType', 'LAT', 'LON', 'SOG'])
df

,geohash,VesselType,LAT,LON,SOG,count,weighted_average_SOG
0,dhvmf,5,27.74131,-82.52868,12.9,1,12.9
1,dhvmf,5,27.74581,-82.52707,13.0,1,13.0
2,dhvmf,5,27.75374,-82.52413,13.0,1,13.0
3,dhvmf,5,27.76212,-82.52299,13.0,1,13.0
4,dhvmf,5,27.76644,-82.52305,13.0,1,13.0
5,dhvmf,5,27.77118,-82.52303,12.9,1,12.9
6,dhvq4,5,27.78906,-82.50863,13.6,2,13.6
7,dhvq4,5,27.79221,-82.49941,13.5,2,13.5
8,dhvq4,5,27.79358,-82.49541,13.5,2,13.5
9,dhvq4,5,27.79520,-82.49082,13.5,2,13.5


In [ ]:
df = df.to_csv(path + 'version3_statistical_speed_model_cargo.csv', index = False)

In [ ]:
new_data = pd.read_csv(path + 'version3_statistical_speed_model_cargo.csv')

In [ ]:
temp = new_data.copy()
speed_data = temp.drop(['geohash','count'], axis = 1)
speed_data.head()

,VesselType,LAT,LON,SOG,weighted_average_SOG
0,5,27.74131,-82.52868,12.9,12.9
1,5,27.74581,-82.52707,13.0,13.0
2,5,27.75374,-82.52413,13.0,13.0
3,5,27.76212,-82.52299,13.0,13.0
4,5,27.76644,-82.52305,13.0,13.0


In [ ]:
scale = StandardScaler()
data = scale.fit_transform(speed_data)

In [ ]:
import joblib

joblib.dump(scale, open(path +'version2_speedmodel_Scaler.joblib', 'wb'))

In [ ]:
X = speed_data.drop(['weighted_average_SOG'], axis = 1)
y = speed_data['weighted_average_SOG']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
reg = LinearRegression()

In [ ]:
vari = reg.fit(X_train, y_train)
print(reg.score(X_train, y_train))

1.0


In [ ]:
pred = reg.predict(X_test)

r2_score = reg.score(X_test, y_test)
print(r2_score)

1.0


In [ ]:
#save the trained model as a joblib string
saved_model = joblib.dump(reg, open(path +'version2_SpeedModel.joblib', 'wb'))

# load the pickled model - reg_from_joblib = joblib.load(saved_model)
# use the loaded pickled model to make predictions - reg_from_joblib.predict(X_test)